In [1]:
#By defauly it is in research older , we need to go to base folder.
import os
%pwd

'c:\\Users\\user\\OneDrive\\Work\\Trails\\chicken-disease-classification\\research'

In [2]:
os.chdir("../")
%pwd
#PWD - Print Working Directory

'c:\\Users\\user\\OneDrive\\Work\\Trails\\chicken-disease-classification'

In [ ]:
#def load_json(path: Path) -> ConfigBox: in this ConfigBox is the return type, if we want we can write our own return type called Entity.

In [3]:
#Create a new file config_entity inside entity and add the entity code. for converting to modular programming.
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
#Later while converting to modular programming ove the below codes to confiuration.py inside the config folder
from chicken_disease_classification.constants import *
from chicken_disease_classification.utils.common import read_yaml,create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig: #Observe here we are using our custom entity as return type
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [6]:
#While converting this to modular programming move this to data_ingestion.py file inside the components folder.
import os
import urllib.request as request
import zipfile
from chicken_disease_classification import logger
from chicken_disease_classification.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
#Creating the pipeline 
#Create a new file config_entity inside entity and add the entity code. for converting to modular programming.
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-05 12:09:16,222: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-05 12:09:16,223: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-05 12:09:16,223: INFO: common: created directory at: artifacts]
[2023-08-05 12:09:16,224: INFO: common: created directory at: artifacts/data_ingestion]
[2023-08-05 12:09:19,022: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 2811873
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "5db458c593cd8487a529cf7a56166be1a5f9a4d9d9365d93fb85333acf622df0"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 566E:2D7A85:50438:7F200:64CDEE8F
Accept-Ranges: bytes
Date: Sat, 05 Aug 2023 06:39:14 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830024-CCU
X-Cache: MI

In [ ]:
#Now we have tried it all here, let's move it to modular programming.
